# Construction et visualisation de réseau

- Récupérer tous les articles sur HAL d'un auteur
- Construction du réseau de co-auteurs
- Représentation

## Chargement des bibliothèques

In [ ]:
#!pip install -r requirements.txt

## Récupération des données

Utilisation de l'API de HAL https://api.archives-ouvertes.fr/docs/search/?schema=fields#fields

In [ ]:
import requests as rq

req = rq.get("http://api.archives-ouvertes.fr/search", params={
    "q": "Emilien Schultz",
    "wt": "json",
    "fl": "title_s,authFullName_s,docid",
    "rows": 400
})
data = req.json()
len(data['response']['docs'])

326

Un élément

In [3]:
docs = data['response']['docs']
docs[1]

{'docid': '3968415',
 'title_s': ['L’approche des controverses par les STS est-elle hégémonique ?'],
 'authFullName_s': ['Nicolas Benvegnu', 'Émilien Schultz']}

Sauvegarder

In [11]:
import pickle
pickle.dump(docs, open("data/hal_emilien_schultz.pkl", "wb"))

## Transformation des données

- Il faut découper chaque auteur
- Nettoyer les noms
- Créer le réseau

In [ ]:
import unicodedata

def remove_accents(text):
    return ''.join(
        char for char in unicodedata.normalize('NFKD', text)
        if not unicodedata.combining(char)
    )

Créer le réseau

In [ ]:
import itertools
import networkx as nx
from ipysigma import Sigma

reseau = nx.Graph()
for element in docs:
    if len(element['authFullName_s'])>10:
        continue
    for auteur in element['authFullName_s']:
        auteur = remove_accents(auteur)
        if reseau.has_node(auteur):
            reseau.nodes[auteur]["weight"] +=1
        else:
            reseau.add_node(auteur,weight=1)
    for p in itertools.combinations(element['authFullName_s'],2):
        p = (remove_accents(p[0]), remove_accents(p[1]))
        if reseau.has_edge(p[0],p[1]):
            reseau.edges[p]["weight"] += 1
        else:
            reseau.add_edge(p[0],p[1], weight=1)
            
nx.write_graphml(reseau, "reseau.graphml")
print(reseau)

Graph with 83 nodes and 275 edges


Visualiser le réseau

In [ ]:
# If needed, enable ipysigma extension
#!jupyter labextension enable ipysigma
#!jupyter labextension enable ipysigma user
#!jupyter labextension enable ipysigma sys-prefix

Sigma(reseau, node_size = "weight")

Sigma(nx.Graph with 83 nodes and 275 edges)